## Import Packages

In [1]:
import requests
import pandas as pd
#from google.cloud import bigquery
from datetime import datetime, timedelta

## Extract data

In [2]:
def fetch_exchange_rates(API_KEY, base_currency, target_currencies, start_date, end_date):

    rates_data = []
    current_date = start_date
    
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        api_url = f"http://api.exchangeratesapi.io/v1/{date_str}?access_key={API_KEY}&base={base_currency}&symbols={','.join(target_currencies)}"
        response = requests.get(api_url)
        data = response.json()
        rates_data.append(data)
        current_date += timedelta(days=1)

    return rates_data

In [37]:
API_KEY = '0fd5492ce37b5afa389fd204e9fb854c'
base_currency = 'EUR'
target_currencies = ['USD', 'GBP', 'CHF']
#end_date = datetime.now().strftime('%Y-%m-%d')
#start_date = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')  # 3 months back
start_date = datetime.now() - timedelta(days=10)
end_date = datetime.now()
exchange_rates = fetch_exchange_rates(API_KEY, base_currency, target_currencies, start_date, end_date)

## Transformation

In [38]:
len(exchange_rates)

11

In [58]:
df = pd.DataFrame(exchange_rates)

In [59]:
# Apply the transformation using lambda
#df[['CURRENCY_CODE', 'RATE']] = 
#df['rates'].apply(lambda x: pd.Series(list(x.items())))
#df = pd.read_csv('data.csv')
#df = df.drop('Unnamed: 0', axis=1)

In [60]:
# Define the unpivot_data function
def unpivot_data(row):
    rows = []
    success = row['success']
    timestamp = row['timestamp']
    historical = row['historical']
    base = row['base']
    date = row['date']
    rates = row['rates']
    for currency_code, exchange_rate in rates.items():
        rows.append([success, timestamp, historical, base, date, currency_code, exchange_rate])
    return pd.DataFrame(rows, columns=['success', 'timestamp', 'historical', 'base', 'date', 'currency_code', 'exchange_rate'])

# Apply the function to the DataFrame using lambda
result_df = df.apply(lambda row: unpivot_data(row), axis=1)

# Concatenate the resulting DataFrames into one
final_df = pd.concat(result_df.tolist(), ignore_index=True)
final_df = final_df.drop(['historical', 'success'], axis=1)
final_df['rate_id'] = range(1, len(final_df) + 1)

In [62]:
final_df

,timestamp,base,date,currency_code,exchange_rate,rate_id
0,1712188799,EUR,2024-04-03,USD,1.083829,1
1,1712188799,EUR,2024-04-03,GBP,0.856604,2
2,1712188799,EUR,2024-04-03,CHF,0.978676,3
3,1712275199,EUR,2024-04-04,USD,1.083656,4
4,1712275199,EUR,2024-04-04,GBP,0.857333,5
5,1712275199,EUR,2024-04-04,CHF,0.977003,6
6,1712361599,EUR,2024-04-05,USD,1.084775,7
7,1712361599,EUR,2024-04-05,GBP,0.858276,8
8,1712361599,EUR,2024-04-05,CHF,0.978146,9
9,1712447999,EUR,2024-04-06,USD,1.084775,10


In [54]:
# currencyrates:
# rate_id (Primary Key): Unique identifier for each rate entry.
# timestamp: Timestamp of the rate data.
# base: The base currency.
# date: The date of the rates.
# currency_code: The code of the currency (e.g., USD, GBP, CHF).
# exchange_rate: The exchange rate for the respective currency on the given date.